In [1]:
import nifgen
import time
import numpy as np
import os
import math

In [28]:
# Outputs a sine wave
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.FUNC
    session.configure_standard_waveform(waveform = nifgen.Waveform.SQUARE, amplitude=1.0, frequency = 7911.16)
    with session.initiate():
        time.sleep(10)

In [35]:
# Outputs a sine wave
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.FUNC
    session.configure_standard_waveform(waveform = nifgen.Waveform.SQUARE, amplitude=1.0, frequency = 7911000.15)
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    with session.initiate():
        time.sleep(10)

In [27]:
import nifgen
import numpy as np
import time

# TTL aligned wave
# Settings
resource_name = "Dev1"
sample_rate = 10_000_000  # 10 MS/s
waveform_freq = 7905.16      # Example frequency for your waveform generation
duration = 63.2e-5         # 100 µs = waveform duration
#duration = 63.6e-6         # 100 µs = waveform duration
num_samples = int(sample_rate * duration)

# Create a square wave manually (since standard waveforms don't support finite ARB retriggering)
t = np.linspace(0, duration, num_samples, endpoint=False)
# Creating a simple square wave (0 or 1 for TTL, or -1 to 1 for generic output)
# Adjust amplitude and offset as needed for your device and desired output
waveform = (np.sign(np.sin(np.linspace(0, 2 * np.pi * waveform_freq * duration, num_samples)))).astype(float)
# If you want 0 to 1, you can do: waveform = (np.sign(np.sin(...)) + 1) / 2
# For now, let's assume -1 to 1 is acceptable or your device scales it.
waveform = waveform.tolist()


# Initialize session
with nifgen.Session(resource_name) as session:
    # Set to ARB mode
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = sample_rate

    # Write waveform to device memory
    wf_handle = session.create_waveform(waveform)

    # Configure trigger
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"

    # Configure the arbitrary waveform to be played
    # The gain and offset depend on your desired output voltage range.
    # For now, we'll assume defaults or 1.0 gain, 0.0 offset.
    session.configure_arb_waveform(wf_handle, gain=1.0, offset=0.0)

    # Set the number of times the waveform should repeat *per trigger*
    # This is the property that controls the "finite" aspect you mentioned.

    # Enable retriggering by setting the repetition mode to continuous
    # and using the start trigger to gate each repetition.
    # The combination of start_trigger_type and waveform_repetitions = 1
    # is what enables finite retriggering of a single waveform.
    # Many NI FGen devices handle retriggering implicitly when a start trigger
    # is configured and the output mode is ARB.

    session.initiate()

    print("Armed. A new waveform will play on each rising TTL pulse at PFI1.")
    time.sleep(15)  # Keep session alive for 10 seconds

Armed. A new waveform will play on each rising TTL pulse at PFI1.


In [9]:
# Chunking waveform for streaming

# Raw waveform
waveform = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"

# Determining allocation size
def allocation_size(waveform):
    total_size_bytes = os.path.getsize(waveform)
    max_chunk_size = 10 * 1024 * 1024 # 200MB

    valid_chunks = set()
    for i in range(1, int(math.isqrt(total_size_bytes))+1):
        if total_size_bytes % i == 0:
            complement = total_size_bytes // i
            if i <= max_chunk_size:
                valid_chunks.add(i)
            if complement <= max_chunk_size:
                valid_chunks.add(complement)
    return(max(valid_chunks) if valid_chunks else None)

# Setting waveform handle and waveform size
waveform_size = allocation_size(r"C:\Users\max\Desktop\NanoStride\waveform_output.bin")


In [10]:
# Loading first chunk for test
first_chunk = open(waveform, "rb").read(allocation_size(waveform))


In [11]:
len(first_chunk)

9536000

In [3]:
with nifgen.Session("Dev1") as session:
    session.clear_arb_memory()

In [6]:
# Testing abitrary waveform

#waveform = np.fromfile(r"C:\Users\max\Desktop\NanoStride\waveform_output.bin", dtype=np.float64)
#waveform_handle = session.create_waveform_from_file_f64(r'C:\Users\max\Desktop\NanoStride\waveform_output.bin', byte_order=nifgen.ByteOrder.LITTLE)


with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 25e6

    num_samples = 100
    waveform = np.sin(np.linspace(0, 2 * np.pi, num_samples)).tolist()
    waveform_handle = session.create_waveform(waveform)
    #waveform_handle = session.streaming_waveform_handle()
    #session.channels["0"].arb_waveform = waveform_handle

    with session.initiate():
        time.sleep(5)

In [19]:
def stream_waveform(waveform_path, chunk_size_bytes):
    with open(waveform_path, "rb") as f:
        first_bytes = f.read(chunk_size_bytes)

    # Convert raw bytes to float64, then to float32
    first_data = np.frombuffer(first_bytes, dtype='<f8').astype(np.float32)

    with nifgen.Session("Dev1") as session:
        session.output_mode = nifgen.OutputMode.SCRIPT
        session.arb_sample_rate = 25e6

        waveform_name = "streamwave"
        num_samples = len(first_data)


        # ✅ Allocate a named waveform
        session.allocate_named_waveform(waveform_name, num_samples)

        # ✅ Write the waveform using the name (not handle)
        session.write_waveform(waveform_name, first_data.tolist())

        # ✅ Write a script that references the named waveform
        session.write_script(f"""
        script main
                generate {waveform_name}
        end script
        """)


        # ✅ Start generation
        with session.initiate():
            session.wait_until_done()

if __name__ == "__main__":
    chunk_size_bytes = 64000 * 1024  # 64 KB
    waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"
    stream_waveform(waveform_path, chunk_size_bytes)


In [22]:
def stream_full_waveform(waveform_path):

    # Determine chunk size and number of chunks
    chunk_size_bytes = allocation_size(waveform_path)
    total_size_bytes = os.path.getsize(waveform_path)
    num_chunks = total_size_bytes // chunk_size_bytes

    with open(waveform_path, "rb") as f:
        for _ in range(num_chunks):
            print(_)
            chunk = f.read(chunk_size_bytes)

            # Ensure chunk size is a multiple of 8 bytes (float64)
            valid_bytes = len(chunk) - (len(chunk) % 8)
            chunk = chunk[:valid_bytes]

            data = np.frombuffer(chunk, dtype='<f8').astype(np.float32)

            with nifgen.Session("Dev1") as session:
                session.output_mode = nifgen.OutputMode.SCRIPT
                session.arb_sample_rate = 25e6

                waveform_name = "streamwave"
                num_samples = len(data)


                # ✅ Allocate a named waveform
                session.allocate_named_waveform(waveform_name, num_samples)

                # ✅ Write the waveform using the name (not handle)
                session.write_waveform(waveform_name, data.tolist())

                # ✅ Write a script that references the named waveform
                session.write_script(f"""
                script main
                    repeat 1
                        generate {waveform_name}
                    end repeat
                end script
                """)


                # ✅ Start generation
                with session.initiate():
                    session.wait_until_done()

if __name__ == "__main__":
    chunk_size_bytes = 64000 * 1024  # 64 KB
    waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"
    stream_full_waveform(waveform_path)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [36]:
# Raw waveform path
waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"

# Determine chunk size and number of chunks
chunk_size_bytes = allocation_size(waveform_path)
total_size_bytes = os.path.getsize(waveform_path)
num_chunks = total_size_bytes // chunk_size_bytes

# Read and print each chunk
with open(waveform_path, "rb") as f:
    for _ in range(num_chunks):
        chunk = f.read(chunk_size_bytes)

        # Ensure chunk size is a multiple of 8 bytes (float64)
        valid_bytes = len(chunk) - (len(chunk) % 8)
        chunk = chunk[:valid_bytes]

        data = np.frombuffer(chunk, dtype='<f8').astype(np.float32)
        print(data)


[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -1. ... -1. -1. -1.]
[-1. -1. -

In [23]:
def stream_full_waveform(waveform_path):
    # Determine chunk size and number of chunks
    chunk_size_bytes = allocation_size(waveform_path)
    total_size_bytes = os.path.getsize(waveform_path)
    num_chunks = total_size_bytes // chunk_size_bytes
    bytes_streamed = 0  # Track total bytes streamed

    with open(waveform_path, "rb") as f:
        for _ in range(num_chunks):
            print(_)
            chunk = f.read(chunk_size_bytes)
            bytes_streamed += len(chunk)

            # Ensure chunk size is a multiple of 8 bytes (float64)
            valid_bytes = len(chunk) - (len(chunk) % 8)
            chunk = chunk[:valid_bytes]

            data = np.frombuffer(chunk, dtype='<f8').astype(np.float32)

            with nifgen.Session("Dev1") as session:
                session.output_mode = nifgen.OutputMode.SCRIPT
                session.arb_sample_rate = 25e6

                waveform_name = "streamwave"
                num_samples = len(data)

                # ✅ Allocate a named waveform
                session.allocate_named_waveform(waveform_name, num_samples)

                # ✅ Write the waveform using the name (not handle)
                session.write_waveform(waveform_name, data.tolist())

                # ✅ Write a script that references the named waveform
                session.write_script(f"""
                script main
                    repeat 1
                        generate {waveform_name}
                    end repeat
                end script
                """)

                # ✅ Start generation
                with session.initiate():
                    session.wait_until_done()

    print(f"Total bytes streamed: {bytes_streamed}")
    print(f"Total file size     : {total_size_bytes}")


In [24]:
waveform_path = r"C:\Users\max\Desktop\NanoStride\waveform_output.bin"
stream_full_waveform(waveform_path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
Total bytes streamed: 1192000000
Total file size     : 1192000000
